In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout, TimeDistributed
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-03-22 00:05:58.511058: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:05:58.517442: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-22 00:05:58.535351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742591158.565162 1361609 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742591158.574308 1361609 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 00:05:58.606551: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=False))

    model.add(Dropout(0.2))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [ ]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = []
USE_DIFF = True
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

if USE_DIFF:
    for feature_name in FEATURES:
        data[feature_name] = data[feature_name] - data.groupby(IDS)[feature_name].shift(1).fillna(0)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,0.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,0.0,-328.0,-228176.0,-62529.0,-32296.0,-21751.0,-25200.0,-41410.0,-27935.0,-5553.0,0.0
2,0,0.0,151.0,684566.0,355662.0,200560.0,59867.0,33547.0,126849.0,51950.0,22614.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-03-22 00:06:03.354278: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 144, 9)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 10)        │           828 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 144, 10)        │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_2 (LTC)                     │ (None, 10)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,641 (10.32 KB)

 Trainable params: 2,641 (10.32 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5436 - auc: 0.6566 - loss: 0.6909 - val_accuracy: 0.5000 - val_auc: 0.4333 - val_loss: 0.6969
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 794ms/step - accuracy: 0.4252 - auc: 0.5398 - loss: 0.7065 - val_accuracy: 0.4667 - val_auc: 0.4000 - val_loss: 0.6941
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 911ms/step - accuracy: 0.4948 - auc: 0.5063 - loss: 0.6950 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 917ms/step - accuracy: 0.4451 - auc: 0.4515 - loss: 0.7077 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 835ms/step - accuracy: 0.4577 - auc: 0.4465 - loss: 0.7041 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 951ms/step - accuracy: 0.4150 - auc: 0.3932 - loss: 0.7135 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 833ms/

1it [25:51, 1551.53s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5780 - auc: 0.6045 - loss: 0.6796 - val_accuracy: 0.5000 - val_auc: 0.5333 - val_loss: 0.6957
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 821ms/step - accuracy: 0.5851 - auc: 0.5940 - loss: 0.6752 - val_accuracy: 0.5667 - val_auc: 0.6000 - val_loss: 0.6895
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 858ms/step - accuracy: 0.5572 - auc: 0.5787 - loss: 0.6784 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6931
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 835ms/step - accuracy: 0.3843 - auc: 0.3597 - loss: 0.7119 - val_accuracy: 0.5000 - val_auc: 0.3333 - val_loss: 0.6933
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 896ms/step - accuracy: 0.5515 - auc: 0.5402 - loss: 0.6885 - val_accuracy: 0.5667 - val_auc: 0.5667 - val_loss: 0.6918
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 920ms/step - accuracy: 0.4957 - auc: 0.4902 - loss: 0.6887 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6927
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 832ms/s

2it [51:51, 1556.51s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.4821 - auc: 0.4469 - loss: 0.7160 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6923
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 843ms/step - accuracy: 0.4270 - auc: 0.3211 - loss: 0.7337 - val_accuracy: 0.5000 - val_auc: 0.5667 - val_loss: 0.6929
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 854ms/step - accuracy: 0.5523 - auc: 0.5916 - loss: 0.6835 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6927
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 893ms/step - accuracy: 0.5595 - auc: 0.5152 - loss: 0.6934 - val_accuracy: 0.5000 - val_auc: 0.6667 - val_loss: 0.6915
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 789ms/step - accuracy: 0.4999 - auc: 0.4917 - loss: 0.6963 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6903
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 792ms/step - accuracy: 0.5657 - auc: 0.5531 - loss: 0.6901 - val_accuracy: 0.6000 - val_auc: 0.6333 - val_loss: 0.6879
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 908ms

4it [1:43:07, 1536.57s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.6190 - auc: 0.4648 - loss: 0.6973 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 878ms/step - accuracy: 0.4753 - auc: 0.5455 - loss: 0.6783 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 846ms/step - accuracy: 0.6277 - auc: 0.4846 - loss: 0.6927 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 989ms/step - accuracy: 0.4009 - auc: 0.4752 - loss: 0.6937 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 898ms/step - accuracy: 0.5058 - auc: 0.5813 - loss: 0.6949 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 826ms/step - accuracy: 0.4793 - auc: 0.4524 - loss: 0.6960 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6932
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 869ms

5it [2:06:21, 1516.37s/it]

CPU times: user 3h 30min 40s, sys: 1h 59s, total: 4h 31min 40s
Wall time: 2h 6min 22s


In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

Epoch 1: TRAIN Accuracy = 0.511 Loss = 0.702 AUC = 0.533
Epoch 1: VAL Accuracy = 0.5 Loss = 0.696 AUC = 0.54
Epoch 2: TRAIN Accuracy = 0.471 Loss = 0.703 AUC = 0.488
Epoch 2: VAL Accuracy = 0.507 Loss = 0.692 AUC = 0.54
Epoch 3: TRAIN Accuracy = 0.526 Loss = 0.692 AUC = 0.538
Epoch 3: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.547
Epoch 4: TRAIN Accuracy = 0.474 Loss = 0.697 AUC = 0.463
Epoch 4: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.527
Epoch 5: TRAIN Accuracy = 0.506 Loss = 0.697 AUC = 0.502
Epoch 5: VAL Accuracy = 0.513 Loss = 0.693 AUC = 0.567
Epoch 6: TRAIN Accuracy = 0.469 Loss = 0.699 AUC = 0.463
Epoch 6: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.567
Epoch 7: TRAIN Accuracy = 0.529 Loss = 0.694 AUC = 0.534
Epoch 7: VAL Accuracy = 0.52 Loss = 0.692 AUC = 0.547
Epoch 8: TRAIN Accuracy = 0.503 Loss = 0.694 AUC = 0.513
Epoch 8: VAL Accuracy = 0.52 Loss = 0.691 AUC = 0.573
Epoch 9: TRAIN Accuracy = 0.563 Loss = 0.689 AUC = 0.541
Epoch 9: VAL Accuracy = 0.52 Loss = 0.691 AUC = 0.573
Epoc